# **Time Series forecasting using classic statistical techniques**

## **Setting up the environment**

In [ ]:
#Libraries needed in this notebook
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import numpy as np
#import timeSeriesMatrix
#import timeSeriesAnalysis
from numpy import loadtxt
from numpy import array

#to track the time for the LSTM training part
!pip install ipython-autotime
%load_ext autotime

taxitrips_df = pd.read_csv("/content/two_years_2015_2016_dataframe.csv")

In [ ]:
from datetime import date
from datetime import timedelta
import datetime

initial_date = datetime.datetime(2015, 1, 1, 0, 0)

def convertDatetoHourIndex(taxiTripTimeStamp, yeartaxitrip):
  taxitripdate = datetime.datetime(year=yeartaxitrip, month=taxiTripTimeStamp[0], day=taxiTripTimeStamp[1], hour=taxiTripTimeStamp[2])
  diff = taxitripdate - initial_date
  return int(diff.total_seconds() / 3600.0)

time: 4.21 ms


In [ ]:
#installing stats libraries

!pip install tbats

!pip install pmdarima

In [ ]:
regions_with_minimum_taxi_trips = [[26, 365168.0], [53, 26872.0], [60, 459962.0], [62, 15995.0], [65, 18050.0], [66, 2536978.0], [67, 21184.0], [68, 59604.0], [72, 750790.0], [73, 96074.0], [75, 155855.0], [76, 716215.0], [77, 1399092.0], [78, 496385.0], [80, 30788.0], [82, 64346.0], [83, 427156.0], [88, 86480.0], [100, 62372.0], [125, 107300.0]]

regions_selected_array = []

for elem in regions_with_minimum_taxi_trips:
  regions_selected_array.append(elem[0])

print("List of region IDs with minimum taxi trips and representing 98% of all taxi flows")
print(regions_selected_array)

List of region IDs with minimum taxi trips and representing 98% of all taxi flows
[26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]
time: 4.7 ms


## **Implementation of the functions**

**Time series retrieving**

In [ ]:
def returnTaxiTripsTensorForCluster(clusterRegionIDsArray, startingIndex, endingIndex, thetaxiTripsTensor , regionsFilteredArray, askingNewFlow = True):
  indexClusterArray = []
  for regionIDvalue in  clusterRegionIDsArray:
    regionIDIndex = timeSeriesAnalysis.returnRegionIndexinArray( regionIDvalue, regionsFilteredArray )
    indexClusterArray.append(regionIDIndex)
  
  k = 0
  if askingNewFlow == False:
    k = 1

  clustertaxiTripsTensor = []
  i = startingIndex
  while i < endingIndex:
    clusterTaxiTripsRow = []
    for regionIDindex in indexClusterArray:
      flow = thetaxiTripsTensor[i][regionIDindex*2 + k]
      clusterTaxiTripsRow.append(flow)
    clustertaxiTripsTensor.append(clusterTaxiTripsRow)
    i = i + 1
  return clustertaxiTripsTensor

#this function returns the time series on which to train and make forecasts
def returnTimeSeriesForRegion(clusterRegionIDsArray, startingIndex, endingIndex, endingTrainIndex, thetaxiTripsTensor , regionsFilteredArray, askingNewFlow = True):
  maxforecastperiod = 24*7*2
  y_timeSeries = returnTaxiTripsTensorForCluster(clusterRegionIDsArray, startingIndex, endingIndex, thetaxiTripsTensor , regionsFilteredArray, askingNewFlow)
  y_training = array(y_timeSeries[:endingTrainIndex])
  y_testing = array(y_timeSeries[endingTrainIndex:endingTrainIndex+maxforecastperiod])

  y_training = y_training.reshape(y_training.shape[0])
  y_testing = y_testing.reshape(y_testing.shape[0])

  return [y_training, y_testing]

time: 16 ms


In [ ]:
def returnMSESumFor(timeSeriesForecast, timeSeriesReal):
  n = len(timeSeriesForecast)
  if len(timeSeriesReal) != n:
    print("error : RMSE , not same sizes arrays")
    return -1
  i = 0 
  mse = 0
  while i < n:
    if timeSeriesForecast[i] < 0:
      timeSeriesForecast[i] = 0
    mse = mse + (timeSeriesForecast[i] - timeSeriesReal[i])**2
    i = i + 1
  return mse

def returnRMSEofTensors(yforecast, y):
  yforecast[yforecast < 0] = 0
  ydifference = (yforecast - y)**2
  rmse = (ydifference.sum()/(y.shape[0]*y.shape[1]))**0.5
  return rmse

def returnRMSEofTensorsForPeriodForecastArray(yforecast, y, periodForecastArray):
  rmseArray = []
  for periodForecast in periodForecastArray:
    rmse = returnRMSEofTensors(yforecast[:periodForecast], y[:periodForecast])
    rmseArray.append(rmse)
  return rmseArray

def returnSumofSquares(yforecast, y):
  yforecast[yforecast < 0] = 0
  ydifference = (yforecast - y)**2
  sumSquares = ydifference.sum()
  return sumSquares

def returnSumofSquaresOfTensorsForPeriodForecastArray(yforecast, y, periodForecastArray):
  sumSquaresArray = []
  for periodForecast in periodForecastArray:
    sumSquares = returnSumofSquares(yforecast[:periodForecast], y[:periodForecast])
    sumSquaresArray.append(sumSquares)
  return sumSquaresArray

time: 21.1 ms


Dataframe building for Prophet

In [ ]:
def returnFullDateArray(initialDateWithTime, timeRangeinHours = 24*365):
  fullDateArray = pd.date_range(pd.Timestamp(initialDateWithTime),
                    periods=timeRangeinHours, freq='h')

  return fullDateArray

def returnDataframeFor(timeseriesArray, fullDateArray):
  df = pd.DataFrame(timeseriesArray)
  df.columns = ["y"]
  df["ds"] = fullDateArray 
  return df

In [ ]:
from tbats import TBATS, BATS
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from fbprophet import Prophet

def forecastWithProphet(dataframeTrainY, forecastingHourRange):
  model = Prophet(yearly_seasonality=True,daily_seasonality=True, weekly_seasonality=True)
  model.fit(dataframeTrainY)
  forecast = model.make_future_dataframe(periods=forecastingHourRange, include_history=False)
  forecast = model.predict(forecast)
  return forecast

def forecastWithTBATS(ytrain, forecastingHourRange):
  # Fit the model
  estimator = TBATS(seasonal_periods=(24, 24*7, 24*365.25))
  model = estimator.fit(ytrain)
  # Forecast 1 day ahead
  yforecasted = model.forecast(steps=forecastingHourRange)
  return yforecasted

def forecastWithSARIMA(ytrain, forecastingHourRange):
  arima_model = auto_arima(ytrain, seasonal=True, m=24)
  yforecasted = arima_model.predict(n_periods=forecastingHourRange)
  return yforecasted

def forecastWithHoltWinters(ytrain, forecastingHourRange):
  model = ExponentialSmoothing(ytrain, trend = "add", damped = True, seasonal="add", seasonal_periods=24).fit()
  yforecasted = model.predict(start=0, end=forecastingHourRange - 1)
  return yforecasted

def makeForecastFor(statisticalmethod, ytrain, forecastingHourRange):

  yforecasted = None

  if statisticalmethod == "TBATS":
    yforecasted = forecastWithTBATS(ytrain, forecastingHourRange)

  if statisticalmethod == "SARIMA":
    yforecasted = forecastWithSARIMA(ytrain, forecastingHourRange)
  
  if statisticalmethod == "Holt":
    yforecasted = forecastWithHoltWinters(ytrain, forecastingHourRange)

  if statisticalmethod == "Prophet":
    #dataframeY = returnDataframeFor(ytrain)
    forecast = forecastWithProphet(ytrain, forecastingHourRange)
    yforecasted = forecast.yhat[:forecastingHourRange]

  return yforecasted

time: 39.8 ms


## **Taxi flows forecasting**

In this part, we apply the statistical methods to the taxi trips considering inter and intra taxi flows of each region.

In [ ]:
#OLD

def runSimulationWith(statisticalMethod, regionIDsToForecastArray, startingIndex, endingIndex, endingIndexTraining, thetaxiTripsTensor, regionsFilteredArray, askingnewflow):
  
  rmse_Forecasting_Period_Arrays = [5, 10, 12, 16, 20, 24, 168, 336]
  sumOfSquaresArray = [0, 0, 0, 0, 0, 0, 0, 0 ]

  for regionIDValue in regionIDsToForecastArray:
    print("region ID ", regionIDValue)
    cluster = [regionIDValue]
    y_train, y_test = returnTimeSeriesForRegion(cluster, startingIndex, endingIndex, endingIndexTraining, thetaxiTripsTensor , regionsFilteredArray, askingnewflow)

    y_forecasted = makeForecastFor(statisticalMethod, y_train)

    j = 0
    for periodToForecast in rmse_Forecasting_Period_Arrays:
      sumofsquares = returnMSESumFor(y_forecasted[:periodToForecast], y_test[:periodToForecast] )
      prevValue = sumOfSquaresArray[j]
      sumOfSquaresArray[j] = prevValue + sumofsquares
      j = j + 1

    print("Sum of squares ", sumOfSquaresArray)
    print()
  
  return sumOfSquaresArray

time: 11.4 ms


In [ ]:
#NEW

def runForecastingSimulation(statisticalMethod, forecastingHourRange, ytrain, ytest, regionID):
  rmse_Forecasting_Period_Arrays = [1, 2, 3, 4, 5, 10, 12, 16, 24, 24*2, 24*3, 24*7, 24*7*2, 24*7*3]
  sumOfSquaresArray = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  rmse_values_array = [] 
  average_ytest_array = []
  relative_rmse_array = []

  yforecasted = makeForecastFor(statisticalMethod, ytrain, forecastingHourRange)

  j = 0
  for periodToForecast in rmse_Forecasting_Period_Arrays:
    sumofsquares = returnMSESumFor(yforecasted[:periodToForecast], ytest[:periodToForecast] )
    sumOfSquaresArray[j] = sumofsquares
    rmse = (sumofsquares/periodToForecast)**0.5
    rmse_values_array.append( rmse )
    average_ytest = np.sum(ytest[:periodToForecast])/periodToForecast
    rmse_to_average_ytest_value = 0
    if average_ytest > 0:
      rmse_to_average_ytest_value = int(100*rmse / average_ytest)
    average_ytest_array.append( int(average_ytest) )

    relative_rmse_array.append(rmse_to_average_ytest_value)
    j = j + 1

  print("Region ID :", regionID)
  print("Forecasting period :", rmse_Forecasting_Period_Arrays)
  print("RMSE values :", rmse_values_array)
  print("Average taxi trips :", average_ytest_array)
  print("Relative RMSE to average taxi trips %:", relative_rmse_array)


time: 25 ms


## **TBATS**

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = True

statistical_method = "TBATS"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

region ID  26
Sum of squares  [70.66545668256768, 2389.1904432404117, 8793.633683982871, 22339.172468010547, 22939.503404466115, 25251.60588174062, 584552.6278134127, 1090431.2984903762]

region ID  53
Sum of squares  [80.00283715277367, 2410.6114488276357, 8831.829180381455, 22387.20047020168, 22993.38832353812, 25333.815340385114, 585900.673078528, 1095907.042935174]

region ID  60
Sum of squares  [676.3433274786127, 8404.938076974775, 14877.762002662737, 28670.983938720878, 34363.8811213683, 44532.71622848938, 1088074.4958949464, 1935695.1171671855]

region ID  62
Sum of squares  [676.5086527235669, 8405.259667294533, 14878.291128160607, 28675.090002780016, 34369.98023555957, 44543.26553901852, 1088181.339779632, 1935991.9504473866]

region ID  65
Sum of squares  [698.3490481344721, 8427.100062705438, 14900.16311945078, 28697.01494028533, 34391.905173064886, 44567.88685554651, 1089565.9356966855, 1938188.7437110578]

region ID  66
Sum of squares  [91432.0092094437, 110802.6972500151

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = False

statistical_method = "TBATS"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for end-flows only")
print(sumOfSquaresArray)

region ID  26
Sum of squares  [13.29209871717448, 5368.797451337953, 9957.675842086097, 22909.77395282843, 32687.83954312939, 32995.12950627032, 174560.91577661238, 312213.155657891]

region ID  53
Sum of squares  [61.63673657179467, 5419.049980836112, 10010.159889878747, 22976.950342185268, 32778.703059512496, 33118.71521511699, 179687.09547152382, 320919.1343529515]

region ID  60
Sum of squares  [592.0536968548342, 6119.238804635754, 11488.192275839358, 37561.05333994167, 62379.586191736584, 91551.25187994062, 1053783.3383132343, 1858348.71585822]

region ID  62
Sum of squares  [725.1000405728827, 6265.107736896477, 11634.758730836404, 37739.49902914838, 62811.67705443555, 92345.04012004653, 1059880.7260608485, 1869719.090143033]

region ID  65
Sum of squares  [738.6596680966385, 6278.757922020322, 11652.430606509333, 37785.53896400887, 62902.44876449577, 92750.0938667681, 1063239.0207624114, 1877050.5302707218]

region ID  66
Sum of squares  [30958.987660387753, 52369.15219943844, 

## **HOLT WINTERS**

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = True

statistical_method = "Holt"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

region ID  26
Sum of squares  [509.0804882394669, 828.3705432902086, 1028.6634564068067, 2903.6895146577144, 8822.407531402701, 23926.922497040596, 1021017.287687128, 2015277.703336217]

region ID  53
Sum of squares  [510.6526053618468, 843.5796513441625, 1083.8621614998804, 2972.287479836729, 8915.782576116999, 24080.11172127603, 1023600.5386581906, 2024206.6274013238]

region ID  60


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [18453.386810775923, 22578.69032345475, 23182.528044261824, 25872.67996621668, 40497.86370067415, 244912.06316304678, 2646579.183598753, 6251841.02597787]

region ID  62
Sum of squares  [18453.844132388695, 22579.150772639685, 23182.98849344676, 25877.162956497094, 40503.22859130739, 244919.1729461863, 2646684.634617977, 6252312.941767896]

region ID  65
Sum of squares  [18455.11820865775, 22580.425733540487, 23184.265014236084, 25878.441443471875, 40504.80149495416, 244923.88921125757, 2649530.9143670877, 6258188.729302942]

region ID  66
Sum of squares  [38489.83675805334, 48821.602072907655, 52589.41888257973, 115067.10944026257, 366982.5390749411, 777650.4521783679, 33749697.81680152, 70581107.7561566]

region ID  67
Sum of squares  [38492.23878843497, 48839.81923702792, 52622.250631032184, 115117.63965408823, 367053.10359123343, 777730.1815203908, 33750511.13333761, 70582588.58940625]

region ID  68


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [38527.25678894959, 48878.28772960097, 52660.72165125378, 115156.35502614266, 367104.82555209624, 777921.0326035903, 33780893.01214656, 70657053.08373706]

region ID  72
Sum of squares  [38657.28607135879, 51262.67549491779, 55953.962006700065, 127341.98378514113, 392252.945759738, 815506.7937207178, 36417304.738982536, 75039453.88283977]

region ID  73


...

Sum of squares  [39659.503851751506, 52338.23836260556, 57435.98078412854, 130605.67960103926, 404314.6099783818, 834861.7363931191, 36806120.534730025, 75787910.72326763]

region ID  76
Sum of squares  [39844.42261891522, 52889.37521699088, 62036.07150658709, 144802.20911970266, 453631.99351586966, 897488.1566900807, 39014177.05182936, 80064870.74289009]

region ID  77
Sum of squares  [41023.677156233265, 55682.78420123392, 65495.136037583485, 171397.06985418714, 537646.759988559, 1049561.4206554096, 53438548.51967315, 110047433.0787147]

region ID  78


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [43072.07909029, 59664.79731811723, 70132.26659014235, 180897.2157806701, 552407.1509904389, 1416778.0000303607, 55355511.10307985, 114096397.11286692]

region ID  80
Sum of squares  [43361.07909029, 59953.79731811749, 70434.26606694511, 181281.21466708058, 552836.1456400263, 1417327.9907099768, 55448162.04110664, 114236989.4422029]

region ID  82
Sum of squares  [43362.0769814052, 59998.794767416046, 70579.26280596197, 181540.21072833196, 553241.1413278849, 1417793.9862549333, 55455030.490058765, 114251389.77923611]

region ID  83
Sum of squares  [43378.05042701439, 64902.042571365244, 77548.81439813667, 200035.76020662053, 605620.732333665, 1471339.5638511837, 56771015.84343767, 117105196.96604012]

region ID  88
Sum of squares  [43379.9480568597, 64988.37408152701, 77807.46733012349, 200345.89204644415, 606083.7297737036, 1471930.3376344473, 56776888.81298585, 117122740.57492548]

region ID  100


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [43386.711116127444, 65291.375869794574, 78130.70376441126, 200818.42778541156, 606656.4572967417, 1472545.0651574852, 57269683.63710783, 117761862.84065777]

region ID  125
Sum of squares  [43406.77332306037, 65964.63132046144, 79160.34840465235, 202377.16879222743, 608703.9951578804, 1479392.4219670217, 57431291.24178143, 118044437.43219416]

Final sum for new-flows only
[43406.77332306037, 65964.63132046144, 79160.34840465235, 202377.16879222743, 608703.9951578804, 1479392.4219670217, 57431291.24178143, 118044437.43219416]
time: 34.5 s


In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = False

statistical_method = "Holt"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

region ID  26


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [71.19152274572903, 4321.936559002658, 8462.677155149646, 31515.790793526125, 64973.72401996674, 65609.30171794398, 580378.0216815466, 1277948.348660149]

region ID  53
Sum of squares  [72.45964534265124, 4323.794564566523, 8468.569122460707, 31533.473010264726, 65049.45229143255, 65904.2312168587, 590196.4689439212, 1297893.9473020309]

region ID  60
Sum of squares  [1787.229277940944, 6368.524155555434, 10742.263768014705, 36583.98211886087, 150967.87834885347, 398732.96397858916, 3430889.1537094824, 7879530.855866762]

region ID  62


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [1907.5439631085503, 6504.911345578137, 10881.747389647559, 36758.90467734422, 151563.6874024884, 401256.1697987744, 3442964.8480993924, 7908008.309084678]

region ID  65
Sum of squares  [1912.5623959788293, 6509.931710844441, 10893.010616373418, 36784.24780505545, 151703.5408095665, 402441.0498079479, 3452080.050954441, 7931586.344301726]

region ID  66
Sum of squares  [13428.466047299178, 19533.759460144116, 25165.864125026917, 106153.71705895473, 259647.6593622372, 1115028.3388572421, 29168126.000017755, 57837532.22856115]

region ID  67
Sum of squares  [13429.689044743394, 19534.993835646255, 25167.298951059103, 106164.26399713969, 259937.43680775506, 1116377.5175797346, 29188325.93622341, 57878799.1220452]

region ID  68
Sum of squares  [13465.701554986947, 19596.005170235443, 25232.309200369273, 106250.27314430523, 261126.36536409284, 1126754.2610146415, 29265444.78704308, 58035837.23896843]

region ID  72
Sum of squares  [14428.750662396385, 24412.309455677874, 3

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [14439.418159150919, 24508.941539498075, 32819.2745763103, 129410.43321922331, 301018.85980754247, 1193127.931566426, 32542588.8706154, 63682023.9788625]

region ID  76
Sum of squares  [15599.191026953706, 26073.60165288579, 40328.567556440765, 142158.4457755581, 319356.83233531273, 1229550.6864275103, 34319814.658388406, 67172848.92354508]

region ID  77
Sum of squares  [16530.670895839863, 36877.26002635756, 52129.49816367798, 156879.43180668942, 382838.3549622258, 1305420.0539701085, 43735285.57163972, 87853694.6012326]

region ID  78
Sum of squares  [16671.632038500607, 39655.81965554828, 55599.5898626276, 161231.2790428626, 530805.7827016041, 1470417.299374993, 45527582.140856735, 91806231.11030021]

region ID  80


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Sum of squares  [16672.665530736846, 39687.90410231874, 55635.17990806823, 161306.29866515484, 530903.860849636, 1470556.134604557, 45605810.65368072, 91917722.52301599]

region ID  82
Sum of squares  [16673.180192862037, 39784.6147782848, 55751.86154585233, 161475.87926772883, 531686.2504302192, 1472864.8501405905, 45648187.83598626, 92002843.8590733]

region ID  83
Sum of squares  [16710.17864208966, 44796.111966094104, 61611.51426463452, 195717.05920673552, 575773.5059718755, 1550001.1061177393, 46346401.43035836, 93464968.52399698]

region ID  88
Sum of squares  [16710.247312237694, 45015.87898574442, 62249.41165994706, 196822.17464562357, 578174.1592990464, 1554836.3275342318, 46398357.498073824, 93587476.20381391]

region ID  100
Sum of squares  [16710.247312559623, 45530.86105740698, 62766.39334199711, 197448.14866874585, 578965.1236013457, 1555913.2755519694, 46928030.039505616, 94421850.45018879]

region ID  125
Sum of squares  [16733.404943419933, 46197.00652582406, 63551.279

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



## **Prophet**

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = True

statistical_method = "Prophet"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

region ID  26


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Sum of squares  [9516.736540840866, 15401.949269022785, 32043.949269022785, 70118.41712895187, 182205.37477389182, 231417.7545047604, 4123816.7545047603, 8498349.75450476]

region ID  53
Sum of squares  [9707.652110982868, 15655.040625384376, 32359.05928289863, 70499.65774994645, 182693.48127474272, 232008.30821758928, 4130237.2721002474, 8518688.515783966]

region ID  60
Sum of squares  [100786.80121275623, 124276.2223835132, 170935.06889498432, 227192.17390674877, 393722.54579198227, 464299.0144081753, 6011761.171120443, 12215673.674870303]

region ID  62
Sum of squares  [100798.19333590833, 124295.34864044248, 170959.68646283777, 227221.17522641626, 393759.5043266424, 464341.6091947775, 6011954.6456304565, 12216060.513703845]

region ID  65
Sum of squares  [100857.47983557163, 124377.98752995045, 171089.02280918605, 227369.55709476862, 393966.75920133, 464565.52047231345, 6015854.217850132, 12224527.784200529]

region ID  66
Sum of squares  [1930823.846637687, 2283275.863676397, 233

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = False

statistical_method = "Prophet"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

region ID  26
Sum of squares  [3392.6251803581245, 15274.442505621795, 29054.442505621795, 62568.52481740806, 76769.64588953575, 77752.39365356867, 1461121.166713277, 3907011.166713277]

region ID  53
Sum of squares  [4062.1970827520154, 16373.076463122941, 30438.066751067196, 64182.82262800605, 78627.6063695123, 79682.1272532882, 1481557.7415557078, 3958504.035467492]

region ID  60
Sum of squares  [187273.0017684236, 304314.35207855084, 395525.80238608154, 460812.3841255511, 484169.1856355303, 536030.3500821474, 5583992.337716914, 12781172.306519695]

region ID  62
Sum of squares  [187942.13294072202, 305392.81455310003, 396855.2570694262, 462455.3699152185, 485909.9706036712, 538136.7810755364, 5602404.628620958, 12821181.088825447]

region ID  65
Sum of squares  [188620.6371246005, 306547.3845953771, 398350.75601044216, 464248.5937402343, 487828.7616648634, 540368.5453860189, 5621848.63623168, 12869825.396651035]

region ID  66
Sum of squares  [1785703.9153176986, 2370857.888883413

## **SARIMA**

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = True

statistical_method = "SARIMA"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

In [ ]:
taxiTripsTensor = np.loadtxt("/content/interintraTaxiTripsTensorSepNov.csv", delimiter=',')

startIndex = 242*24
endIndex = 333*24
endIndexTraining = 61*24

asking_new_flow = False

statistical_method = "SARIMA"

regionIDsToForecast_array = [26, 53, 60, 62, 65, 66, 67, 68, 72, 73, 75, 76, 77, 78, 80, 82, 83, 88, 100, 125]

sumOfSquaresArray = runSimulationWith(statistical_method, regionIDsToForecast_array, startIndex, endIndex, endIndexTraining, taxiTripsTensor, regions_selected_array, asking_new_flow)
  
print("Final sum for new-flows only")
print(sumOfSquaresArray)

## **Prophet 2**

In [ ]:

#taxitrips_df

start_training_index = 0
end_training_index = convertDatetoHourIndex([11, 2, 8], 2016)

forecastHourRange = 24*7*3
regionID = "66_new"
statistical_method = "Prophet"

period_range = end_training_index - start_training_index 

fullDateArray = pd.date_range(pd.Timestamp('2015-01-01'),
                    periods=end_training_index, freq='h')

data = array(taxitrips_df[regionID][:end_training_index])
data = data.reshape(data.shape[0],)

y_train = pd.DataFrame(data)
y_train.columns = ["y"]
y_train["ds"] = fullDateArray 

y_test =array( taxitrips_df[regionID][end_training_index:end_training_index+forecastHourRange].copy())
y_test = y_test.reshape(y_test.shape[0],)

runForecastingSimulation(statistical_method, forecastHourRange, y_train, y_test, regionID)

Region ID : 66_new
Forecasting period : [1, 2, 3, 4, 5, 10, 12, 16, 24, 48, 72, 168, 336, 504]
RMSE values : [638.6784253479194, 581.4933344664943, 477.55838895759274, 479.3104536567856, 588.4412193845812, 640.0674428098532, 864.6592801235336, 776.1637781589535, 685.2427797679645, 611.5678459071495, 728.9880206010267, 634.1424725617571, 553.0948245042828, 538.0166930856607]
Average taxi trips : [1156.0, 1130.5, 1023.0, 1003.75, 1021.4, 1084.8, 1208.25, 1095.125, 939.875, 872.75, 946.1666666666666, 801.8690476190476, 737.5654761904761, 717.9702380952381]
Relative RMSE to average taxi trips %: [55.24899873251898, 51.43682746275933, 46.68214945822021, 47.75197545771214, 57.611241373074336, 59.00326722067231, 71.56294476503484, 70.874446127972, 72.90786325500355, 70.07365750869658, 77.04647038235267, 79.0830465952872, 74.98925076604942, 74.93579323190458]
time: 10.8 s


## **Holt Winters**

In [ ]:
start_training_index = 0
end_training_index = convertDatetoHourIndex([11, 2, 8], 2016)

forecastHourRange = 24*7*3
regionID = "66_new"
statistical_method = "Holt"

y_train = array(taxitrips_df[regionID][:end_training_index].copy())
y_train = y_train.reshape(y_train.shape[0],)

y_test = array(taxitrips_df[regionID][end_training_index:end_training_index+forecastHourRange].copy())

y_test = y_test.reshape(y_test.shape[0],)

runForecastingSimulation(statistical_method, forecastHourRange, y_train, y_test, regionID)

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Region ID : 66_new
Forecasting period : [1, 2, 3, 4, 5, 10, 12, 16, 24, 48, 72, 168, 336, 504]
RMSE values : [232.20326696855716, 474.2816108365533, 813.5307207291755, 850.4692611704675, 777.8305636090747, 819.017107607831, 1002.0232741287575, 871.2961541667802, 733.3705695545865, 700.7734185898412, 734.2123784072962, 695.4405159815828, 694.6610598710912, 689.0840284343184]
Average taxi trips : [1156.0, 1130.5, 1023.0, 1003.75, 1021.4, 1084.8, 1208.25, 1095.125, 939.875, 872.75, 946.1666666666666, 801.8690476190476, 737.5654761904761, 717.9702380952381]
Relative RMSE to average taxi trips %: [20.086787800048196, 41.9532605782002, 79.52401962162028, 84.72919164836539, 76.15337415401163, 75.49936463936496, 82.93178349917298, 79.56134269300584, 78.02852183051859, 80.29486320135676, 77.5986307987278, 86.7274423481642, 94.18296846796758, 95.97668425120875]
time: 6.82 s


## **TBATS**

In [ ]:
start_training_index = 0
end_training_index = convertDatetoHourIndex([11, 2, 8], 2016)

forecastHourRange = 24*7*3
regionID = "66_new"
statistical_method = "TBATS"

y_train = array(taxitrips_df[regionID][:end_training_index].copy())
y_train = y_train.reshape(y_train.shape[0],)

y_test = array(taxitrips_df[regionID][end_training_index:end_training_index+forecastHourRange].copy())

y_test = y_test.reshape(y_test.shape[0],)

runForecastingSimulation(statistical_method, forecastHourRange, y_train, y_test, regionID)

Region ID : 66_new
Forecasting period : [1, 2, 3, 4, 5, 10, 12, 16, 24, 48, 72, 168, 336, 504]
RMSE values : [78.20915016986419, 62.498095612936886, 127.7088602285604, 110.66830214527661, 109.4843131524924, 202.21447329587411, 331.9057950499188, 298.080858608384, 332.66303305971303, 258.69332811101157, 276.91262651025755, 290.8563208801537, 250.0816643723843, 236.67086397565654]
Average taxi trips : [1156, 1130, 1023, 1003, 1021, 1084, 1208, 1095, 939, 872, 946, 801, 737, 717]
Relative RMSE to average taxi trips %: [6, 5, 12, 11, 10, 18, 27, 27, 35, 29, 29, 36, 33, 32]
time: 1h 24min 39s


In [ ]:
start_training_index = 0
end_training_index = convertDatetoHourIndex([11, 2, 8], 2016)

forecastHourRange = 24*7*3
regionID = "66_new"
statistical_method = "TBATS"

y_train = array(taxitrips_df[regionID][:end_training_index].copy())
y_train = y_train.reshape(y_train.shape[0],)

y_test = array(taxitrips_df[regionID][end_training_index:end_training_index+forecastHourRange].copy())

y_test = y_test.reshape(y_test.shape[0],)

runForecastingSimulation(statistical_method, forecastHourRange, y_train, y_test, regionID)

Region ID : 66_new
Forecasting period : [1, 2, 3, 4, 5, 10, 12, 16, 24, 48, 72, 168, 336, 504]
RMSE values : [64.69700080825874, 54.54762215547685, 115.73701178293, 100.90400348155951, 111.1416529863778, 204.32994495873257, 334.2708144702242, 299.6448969975132, 335.0196531432973, 260.3215484478436, 278.33585153457034, 291.4280905266571, 252.982618603687, 244.1084241171384]
Average taxi trips : [1156, 1130, 1023, 1003, 1021, 1084, 1208, 1095, 939, 872, 946, 801, 737, 717]
Relative RMSE to average taxi trips %: [5, 4, 11, 10, 10, 18, 27, 27, 35, 29, 29, 36, 34, 33]
time: 1h 42min 57s
